In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        print(filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV,train_test_split

# **Training Data and Testing Data**

In [ ]:
store = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")
feature = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip")
train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip")
test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip")
sample = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip")

In [ ]:
df_feature = pd.merge(store,feature,on=['Store'])
df_feature.info(),train.info()

In [ ]:
df_feature.Date = pd.to_datetime(df_feature.Date)
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

As Date column would not be understandable by model, we should lable Date by week and year before creating test data and train data

In [ ]:
df_feature['Week'] = df_feature.Date.dt.week 
df_feature['Year'] = df_feature.Date.dt.year

In [ ]:
df_train = pd.merge(train,df_feature,on=["Store","Date","IsHoliday"])
df_test = pd.merge(test,df_feature,on=["Store","Date","IsHoliday"])

In [ ]:
df_train.head()

# **EDA**

Check null value percentage

In [ ]:
df_train.isnull().mean()

There is lot of null value in Markdown1-5. We will see if we could not find any relation, will drop them

**Lets see if we can find any relation for store type**

In [ ]:
#sns.barplot(df_store.Type,df_store.Size,data=df_store)
plt.figure(figsize=(10,10))
sns.lineplot(df_train.Store,df_train.Size,data=df_train,hue=df_train.Type)


In [ ]:
plt.figure(figsize=(10,10))
sns.lineplot(df_train['Store'],df_train['Weekly_Sales'],hue=df_train['Type'])

By looking at above graph, we can conclude that A>B>C in respective of sales and size 

In [ ]:
df_train['Type'] = df_train['Type'].map({'A':3,'B':2,'C':1})
df_test['Type'] = df_test['Type'].map({'A':3,'B':2,'C':1})

In [ ]:
fig = plt.figure(figsize=(20,8))
plt.subplot(3,2,1)
sns.scatterplot(df_train['MarkDown1'],df_train['Weekly_Sales'])
plt.subplot(3,2,2)
sns.scatterplot(df_train['MarkDown2'],df_train['Weekly_Sales'])
plt.subplot(3,2,3)
sns.scatterplot(df_train['MarkDown3'],df_train['Weekly_Sales'])
plt.subplot(3,2,4)
sns.scatterplot(df_train['MarkDown4'],df_train['Weekly_Sales'])
plt.subplot(3,2,5)
sns.scatterplot(df_train['MarkDown5'],df_train['Weekly_Sales'])

Could not find any relation, will drop markdowns

In [ ]:
df_train.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'],axis=1,inplace=True)
df_test.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'],axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df_train.corr(),annot=True)

(Fuel_price, year) and (type, size) are related.
Conclusion - Drop Fuel_Price

In [ ]:
df_train.drop(['Fuel_Price'],axis=1,inplace=True)
df_test.drop(['Fuel_Price'],axis=1,inplace=True)


Relation of IsHoliday ?


In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(df_train['Store'],df_train['Weekly_Sales'],hue=df_train['IsHoliday'])

In [ ]:
sns.boxplot(df_train['IsHoliday'],df_train['Weekly_Sales'])

We can see Sale is highly impacted by Holiday

In [ ]:
plt.figure(figsize=(12,5))
sale_2010 = df_train[df_train.Year==2010].groupby('Week')['Weekly_Sales'].sum()
sale_2011 = df_train[df_train.Year==2011].groupby('Week')['Weekly_Sales'].sum()
sale_2012 = df_train[df_train.Year==2012].groupby('Week')['Weekly_Sales'].sum()
plt.subplot(2,1,1)
sns.lineplot(sale_2010.index,sale_2010.values)
sns.lineplot(sale_2011.index,sale_2011.values)
sns.lineplot(sale_2012.index,sale_2012.values)
plt.subplot(2,1,2)
sns.lineplot(df_train.Week,df_train.IsHoliday)

It indicates that IsHoliday is highly correlated with Weekly Sales
Because of this, we should do cross validation and ensemble techniques

In [ ]:
df_train['IsHoliday']=pd.get_dummies(df_train['IsHoliday'],drop_first=True)
df_test['IsHoliday']=pd.get_dummies(df_test['IsHoliday'],drop_first=True)

Lets see the performance of departments

In [ ]:
weekly_sales = df_train.groupby(df_train['Dept'])['Weekly_Sales'].mean()
plt.figure(figsize=(25,8))
sns.barplot(weekly_sales.index, weekly_sales.values)

We can see some departments are not performing well

Lets look at Temperature, CPI, Unemployment

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(3,2,1)
sns.scatterplot('Temperature','Weekly_Sales',data=df_train)
plt.subplot(3,2,2)
sns.distplot(df_train['Temperature'])
plt.subplot(3,2,3)
sns.scatterplot('CPI','Weekly_Sales',data=df_train)
plt.subplot(3,2,4)
sns.distplot(df_train['CPI'])
plt.subplot(3,2,5)
sns.scatterplot('Unemployment','Weekly_Sales',data=df_train)
plt.subplot(3,2,6)
sns.distplot(df_train['Unemployment'])


In [ ]:
df_train.drop(['Temperature','Unemployment','CPI'],axis=1,inplace=True)
df_test.drop(['Temperature','Unemployment','CPI'],axis=1,inplace=True)

In [ ]:
test_Date=df_test['Date']
df_train.drop(['Date'],axis=1,inplace=True)
df_test.drop(['Date'],axis=1,inplace=True)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df_train.drop(['Weekly_Sales'],axis=1),df_train['Weekly_Sales'],test_size=0.3)

In [ ]:
clf = XGBRegressor()
xgbR = clf.fit(x_train,y_train)
y_pred = xgbR.predict(x_test)

In [ ]:
xgbR.score(x_test,y_test)

In [ ]:
sample['Weekly_Sales']=xgbR.predict(df_test)

In [ ]:
sample.to_csv('submission.csv',index=False)

In [ ]:
from  sklearn.metrics import mean_squared_error

In [ ]:
print((mean_squared_error(y_train,xgbR.predict(x_train)))**(1/2))
print((mean_squared_error(y_test,y_pred))**(1/2))

In [ ]:
clf = RandomForestRegressor()
clf.fit(x_train,y_train)

In [ ]:
print(clf.score(x_test,y_test))
print((mean_squared_error(y_train,clf.predict(x_train)))**(1/2))
print((mean_squared_error(y_test, clf.predict(x_test)))**(1/2))


In [ ]:
clf1 = DecisionTreeRegressor()
clf1.fit(x_train,y_train)
print(clf1.score(x_test,y_test))
print((mean_squared_error(y_train,clf1.predict(x_train)))**(1/2))
print((mean_squared_error(y_test, clf1.predict(x_test)))**(1/2))

In [ ]:
clf.get_params()